In [5]:
!pip install flask flask-cors pymongo geopy transformers


  Obtaining dependency information for flask-cors from https://files.pythonhosted.org/packages/56/07/1afa0514c876282bebc1c9aee83c6bb98fe6415cf57b88d9b06e7e29bf9c/Flask_Cors-5.0.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pymongo from https://files.pythonhosted.org/packages/02/68/b71c4106d03eef2482eade440c6f5737c2a4a42f6155726009f80ea38d06/pymongo-4.10.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for geopy from https://files.pythonhosted.org/packages/e5/15/cf2a69ade4b194aa524ac75112d5caac37414b20a3a03e6865dfe0bd1539/geopy-2.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for dnspython<3.0.0,>=1.16.0 from https://files.pythonhosted.org/packages/68/1b/e0a87d256e40e8c888847551b20a017a6b98139178505dc7ffb96f04e954/dnspython-2.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for geographiclib<3,>=1.52 from https://files.pythonhosted.org/packages/9f/5a/a26132406f1f40cf51ea349a5f11b0a46cec02a2031ff82e391c2537

In [8]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Complete Code for CivicGuardian Application

# Import necessary libraries
from flask import Flask, request, jsonify
from flask_cors import CORS
from pymongo import MongoClient
from geopy.geocoders import Nominatim
from transformers import pipeline
import datetime

# Initialize Flask app and MongoDB client
app = Flask(__name__)
CORS(app)

# MongoDB connection
client = MongoClient("mongodb://localhost:27017/")
db = client.civicguardian
db_issues = db.issues

# AI Model for categorization (using Hugging Face pipeline)
classifier = pipeline("text-classification", model="distilbert-base-uncased")

# Geolocation setup
geolocator = Nominatim(user_agent="civicguardian")

# Endpoint: Report an Issue
@app.route('/report', methods=['POST'])
def report_issue():
    data = request.json
    
    # Extract data
    description = data.get('description')
    image_url = data.get('image_url', None)
    location = data.get('location', None)
    
    # Categorize issue using AI model
    category = classifier(description)[0]['label']
    
    # Geocode location if provided
    if location:
        lat, lon = location['latitude'], location['longitude']
        location_name = geolocator.reverse((lat, lon)).address
    else:
        location_name = "Unknown"

    # Save issue to database
    issue = {
        "description": description,
        "category": category,
        "image_url": image_url,
        "location": {
            "name": location_name,
            "latitude": lat if location else None,
            "longitude": lon if location else None
        },
        "priority": calculate_priority(description),
        "status": "Reported",
        "timestamp": datetime.datetime.now()
    }

    db_issues.insert_one(issue)

    return jsonify({"message": "Issue reported successfully", "issue_id": str(issue['_id'])}), 201

# Helper function: Calculate priority
# Basic example using keywords
# In production, use an ML model for priority scoring
PRIORITY_KEYWORDS = {"urgent": 10, "critical": 8, "severe": 7}
def calculate_priority(description):
    priority_score = 0
    for keyword, score in PRIORITY_KEYWORDS.items():
        if keyword in description.lower():
            priority_score += score
    return min(priority_score, 10)  # Max priority score of 10

# Endpoint: Get All Issues (for Municipality Dashboard)
@app.route('/issues', methods=['GET'])
def get_issues():
    status_filter = request.args.get('status', None)
    query = {"status": status_filter} if status_filter else {}

    issues = list(db_issues.find(query, {"_id": 0}))
    return jsonify(issues)

# Endpoint: Update Issue Status
@app.route('/issues/<issue_id>/status', methods=['PUT'])
def update_issue_status(issue_id):
    new_status = request.json.get('status')

    result = db_issues.update_one({"_id": issue_id}, {"$set": {"status": new_status}})
    
    if result.matched_count == 0:
        return jsonify({"message": "Issue not found"}), 404
    return jsonify({"message": "Status updated successfully"})

# Run the app
if __name__ == '__main__':
    app.run(debug=True)


ImportError: cannot import name 'pipeline' from 'transformers' (C:\Users\harsh\anaconda3\Lib\site-packages\transformers\__init__.py)